<a href="https://colab.research.google.com/github/NiklasElsaesser/FaceRecognition/blob/main/FaceBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project "Bias in Face-Detection"


For labeling, the pictures have been structured in a folder structure which looks like this:

dataset/\
├── men/\
│   ├── happy\
│   │   └── \
│   └── neutral/\
│       ├── neutral_jan1.jpg\
│       ├── neutral_niklas2.jpg \
│       └── ...\
└── women/\
    ├── happy/\
    │   ├── happy_woman1.jpg\
    │   ├── happy_woman2.jpg\
    │   └── ...\
    └── neutral/\
        └── \


Installing necessary libraries
1.   OpenCV to preprocess the data (pictures)  
2.   numpy to organize the data and link it with the labels



In [6]:
!pip install opencv-contrib-python
!pip install numpy as np
!pip install tensorflow
!pip install wandb

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=81cd4c3580c12f9fd53cba44e56a7195061a5a1bceeeed12de6cf173f49b1f9b
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


Import the W&B library and initialize your W&B project at the beginning of your Colab notebook:

In [ ]:
import wandb
wandb.init(project="emotion-recognition-project")

Mounting a Google Drive where the pictures are uploaded from.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now the pictures get loaded and preprocessed.

In [3]:
import os
import cv2
import numpy as np

data = []
labels = []

genders = ["men", "women"]
emotions = ["happy", "neutral"]

for gender in genders:
    for emotion in emotions:
        folder_path = f'/content/drive/MyDrive/Faces/Dataset/{gender}/{emotion}/'
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (48, 48))
            data.append(img)
            # Create labels by combining gender and emotion indices
            label = genders.index(gender) * len(emotions) + emotions.index(emotion)
            labels.append(label)

data = np.array(data)
data = data.reshape((data.shape[0], 48, 48, 1))
labels = np.array(labels)


Now Splitting the Data into a training and test set in an 80 - 20 config.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

Building and training of the model.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(genders) * len(emotions), activation='softmax'))  # Output layer with appropriate number of units

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

wandb_callback = wandb.keras.WandbCallback()

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

checking for loss and accuracy.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.2f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Saving the trained model to the GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model to Google Drive
model.save('/content/drive/MyDrive/Faces/Dataset/emotion_model.h5')

allow for the input of a new picture on which the algorithm wasnt trained to test for the bias of the algorithm.

In [ ]:
from google.colab import files
import numpy as np
import cv2

# Load the trained model
model_path = '/content/drive/MyDrive/Faces/Dataset/emotion_model.h5'
model = load_model(model_path)

# Function to preprocess uploaded images
def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48, 48))
    img = np.expand_dims(img, axis=0)
    img = img.reshape((1, 48, 48, 1))
    return img

# Function to predict emotion based on the uploaded image
def predict_emotion(file_path):
    preprocessed_img = preprocess_image(file_path)
    prediction = model.predict(preprocessed_img)
    gender_index, emotion_index = np.unravel_index(np.argmax(prediction, axis=None), prediction.shape)
    gender = genders[gender_index]
    emotion = emotions[emotion_index % len(emotions)]
    return gender, emotion

# Upload your own picture
uploaded = files.upload()

# Process and predict emotions for the uploaded image(s)
for file_name in uploaded.keys():
    file_path = '/content/' + file_name #add correct file path
    gender, emotion = predict_emotion(file_path)
    print(f"Predicted Gender: {gender.capitalize()}, Predicted Emotion: {emotion.capitalize()}")
